Get a JSON file recording all the stations in the UK and Ireland, and
their lat/long coordinates, then save as a CSV file for Power BI.

Based on data from:

*   Trainline EU (Great Britain):
    https://github.com/trainline-eu/stations

*   Wikipedia (Ireland/Northern Ireland):
    https://en.wikipedia.org/wiki/List_of_railway_stations_in_Ireland

In [13]:
import csv
import gzip # compress file
import io # dealing with various types of Import and Output 
import json # import the built in json module that provides functions and classes to work with JSON data
import re # provides functions and methods to perform pattern-based matching and manipulation of strings using regular expressions
from urllib.request import urlretrieve # he urlretrieve function is used to retrieve data from a URL and save it locally. It's part of Python's standard library and is useful for downloading files from the internet programmatically.
import bs4

In [14]:
# Generate every row in the Trainline EU database
def get_trainline_csv_rows():
    filename, _ = urlretrieve(
        "https://raw.githubusercontent.com/trainline-eu/stations/master/stations.csv"
    )

    with open(filename, encoding='utf-8') as infile:
        reader = csv.DictReader(infile, delimiter=";")

        for row in reader:
            yield row

In [15]:
# Generate every row from the Wikipedia's page named "List of railway stations in Ireland"
def get_wikipedia_rows():
    filename, _ = urlretrieve(
        "https://en.wikipedia.org/wiki/List_of_railway_stations_in_Ireland"
    )

    with open(filename, encoding='utf-8') as infile:
        soup = bs4.BeautifulSoup(infile.read(), "html.parser")

        table = soup.find("table", attrs={"class": "wikitable"})

        rows = iter(table.find_all("tr"))

        field_names = [th_tag.text.strip() for th_tag in next(rows).find_all("th")]
        print("Field names from Wikipedia:", field_names)

        for row in rows:
            yield dict(
                zip(field_names, [td_tag.text.strip() for td_tag in row.find_all("td")])
            )

In [16]:
# Format the full address from available information
def format_address(row):
    address_parts = [row.get("name"), row.get("city"), row.get("country")]
    return ", ".join(part for part in address_parts if part)

In [17]:
# Get only the stations in Ireland (IE)
if __name__ == "__main__":
    stations = []

    for row in get_trainline_csv_rows():
        if row["country"] != "IE":
            continue

        coords = [row["longitude"], row["latitude"]]

        if coords == ["", ""]:
            print(f"⚠️ No coordinates for {row['name']}")
            from pprint import pprint
            pprint(row)
            continue

        stations.append({
            "name": row["name"],
            "address": format_address(row),
            "latitude": row["latitude"],
            "longitude": row["longitude"]
        })

    for row in get_wikipedia_rows():
        # Filter for stations in both Northern Ireland and Republic of Ireland
        if row.get("Location") not in {"Northern Ireland", "Republic of Ireland"}:
            continue

        names = [row.get("Irish name"), row.get("English Name")]

        if not names[0] or not names[1]:
            continue

        names = [name.strip() for name in names]

        long_lat_coords = row["Coordinates"].split("/")[-1].strip()
        match = re.match(
            r"^(?P<longitude>-?\d+\.\d+); (?P<latitude>-?\d+\.\d+)$", long_lat_coords
        )
        assert match is not None, long_lat_coords

        stations.append({
            "name": "/".join(names),
            "address": "/".join(names) + ", " + row.get("Location"),
            "latitude": match.group("latitude"),
            "longitude": match.group("longitude")
        })

    with open("stations.csv", "w", newline='', encoding='utf-8') as csvfile:
        fieldnames = ["name", "address", "latitude", "longitude"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for station in stations:
            writer.writerow(station)

    print(" stations.csv created successfully")

Field names from Wikipedia: ['Irish name', 'English name', 'Coordinates', 'Station code', 'Station ID', 'Location']
 stations.csv created successfully
